In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import re
import requests
import urllib.parse

Get Links to Pages of All Scripts

In [2]:
all_url = "https://imsdb.com/all-scripts.html"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(all_url)
container = driver.find_elements(By.XPATH, '//td[@valign="top"]')[2]

script_page_links = container.find_elements(By.TAG_NAME, "p")

In [ ]:
n_row = len(script_page_links)
link_df = pd.DataFrame({'title': [''] * n_row, 'page_link': [''] * n_row, 'writer': [''] * n_row, 'script_link': [''] * n_row})

In [ ]:
for i in range(len(script_page_links)):
    link_df["title"][i] = script_page_links[i].find_elements(By.XPATH, ".//a")[0].text
    link_df["page_link"][i] = script_page_links[i].find_elements(By.XPATH, ".//a")[0].get_attribute("href")
    link_df["writer"][i] = script_page_links[i].find_elements(By.XPATH, ".//i")[0].text

link_df

Scrape Scripts Links

In [ ]:
for i in range(len(script_page_links)):
    driver.get(link_df["page_link"][i])
    try:
        script_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/scripts/")]')
        if script_links:
            link_df["script_link"][i] = script_links[0].get_attribute("href")
        else:
            link_df["script_link"][i] = "no script"
    except IndexError:
        link_df["script_link"][i] = "no script"

In [ ]:
link_df.to_csv("link_df.csv")

Get Scripts

In [ ]:
link_df = pd.read_csv("link_df.csv", index_col=0)
link_df = link_df[link_df["script_link"] != "no script"]
link_df.reset_index(drop=True, inplace=True)
link_df["script_downloaded"] = ""
link_df

In [6]:
for i in range(915, len(link_df)):
    driver.get(link_df["script_link"][i])

    try:
        script_elements = driver.find_elements(By.TAG_NAME, "pre")
        if script_elements:
            link_df["script_downloaded"][i] = "Yes"

            script = script_elements[0].text
            file_path = "Scripts/" + link_df["title"][i] + ".txt"
            with open(file_path, "w") as file:
                file.write(script)
        else:
            link_df["script_downloaded"][i] = "No"
    except IndexError:
        link_df["script_downloaded"][i] = "No"
        

    

In [7]:
link_df.to_csv("script_ref.csv")